[//]: #![erdos-logo](risecamp/erdos-pylot-risecamp-logo.png)
<img src="risecamp/erdos-pylot-risecamp-logo.png" alt="erdos-logo" style="width: 800px;"/>

# Architecture of an Autonomous Vehicle

[//]: #![Pylotpipeline](./doc/source/images/pylot.png)
<img src="doc/source/images/pylot.png" alt="Pylot Pipeline" style="width: 500px;"/>
<center><b>Figure 1:</b> Architecture of a state-of-the-art autonomous vehicle.</center>

An autonomous vehicle (AV) is typically equipped with multiple instances of sensor including cameras and LiDARs that operate at different frequencies and generate approximately 1GB/s of data.

A state-of-the-art AV pipeline comprises of five modules, each implemented using several components:
1. __Perception__: Recognizes the scene by detecting and classifying objects, lanes, and traffic signs.
2. __Localization__: Provides the location of the vehicle with decimeter-level accuracy.
3. __Prediction__: Estimates how other agents would move in the environment.
4. __Planning__: Generates trajectories for the AV to follow.
5. __Control__: Physically operates the AV by computing and applying steering and throttle commands.


## Tutorial Overview

In this tutorial, we will be using [Pylot](https://github.com/erdos-project/pylot), our state-of-the-art AV pipeline, to safely drive at various speeds in the scenario depicted below. This emergency scenario tests that the vehicle can safely avoid a collision with a pedestrian that enters the vehicle's path from behind a truck parked in the oncoming lane.
<img src="risecamp/scenario.png" alt="risecamp-scenario" style="width: 800px;"/>
<center><b>Figure 2:</b> Emergency scenario used in the tutorial</center>

Specifically, we will focus on the __*planning*__ module of the vehicle to see how changes in the driving environment require the AV to utilize different hyperparameters to ensure safety.

[Exercise 1](#exercise1) walks through the development of the __*planning*__ component, and executes it at a fixed driving speed in the emergency scenario.

[Exercise 2](#exercise2) stresses the planning component developed in [Exercise 1](#exercise1) by increasing the driving speed to see when the configuration becomes unsafe.

[Exercise 3](#exercise3) shows the importance of adjusting the pipeline with driving speed. By tweaking two hyperparameters of the planning component we can help the vehicle ensure safety at higher speeds.

<a id='exercise1'></a>
# Exercise 1: Getting started with Pylot!
![perception](risecamp/perception-crop.gif)
<center><b>Figure 3:</b> Demo of Pylot executing using the CARLA simulator. In the video we show the output of Pylot's obstacle detector.</center>

__GOAL__: This exercise walks through the development of a __*planning*__ component, and its execution inside the [Pylot](https://github.com/erdos-project/pylot) pipeline at a fixed speed.

ERDOS models the AV pipeline as a directed dataflow graph where developers implement the components of the pipeline as __*operators*__ that communicate with each other through typed __*streams*__. Operators can request callbacks upon receipt of messages on a stream, and send the computed results to other operators via a different stream. For example, a __*planning*__ component computes the trajectory of the AV based on the predictions sent by the __*prediction*__ module, and sends it to the __*control*__ module to physically operate the AV.

The following cell provides a skeleton implementation of a `PlanningOperator` that receives a `message` from the `PredictionOperator`. Upon receipt of the message, the `on_prediction_update` method is invoked which retrieves new predictions from the message for the planning algorithm to use. However, the initial implementation returns an empty list of predictions, which incorrectly notifies the planner that there are no obstacles to plan around. 

We will now fix the implementation of the `Planner` by correctly using the received predictions. To do so, please complete the following tasks:
1. Run the following two cells *without changing the code* to visualize the effects of no predictions on the AV.
2. Instead of returning the empty list, return the correct predictions by retrieving them from the `message` using `message.predictions`. 
3. Run the following two cells again with your fixed code to see how the AV drives! 

In [ ]:
from pylot.planning.planning_operator import PlanningOperator

# TODO(1): Run this cell and the next one without any changes to the code 
#          to see the effect of no predictions on on the planner.

class Planner(PlanningOperator):
    def on_prediction_update(self, message):
        # TODO: Instead of the empty list, return `message.predictions`.
        return []

    
# Run the simulator and the scenario (Don't change!)
import pylot.risecamp 
simulator = pylot.risecamp.start_simulator()
scenario = pylot.risecamp.setup_scenario()
print("Please execute the next cell now.")

In [ ]:
# Helper function to start the Pylot pipeline.
video_name = pylot.risecamp.start_pylot(planner=Planner)

# Functions to reset the scenario and the simulator. (Don't change!)
simulator.reset()
scenario.reset()

# Visualize the drive.
from IPython.display import Video
Video(video_name, html_attributes="loop autoplay")

<details>
    <summary>Click here for the solution</summary>
    <img src="risecamp/sol1.png" alt="solution1" style="width: 640px;"/>
</details>

<b>NOTE:</b> Please wait a few seconds for the above cell to start the visualization. If the visualization does not start within 20 seconds, please restart the kernel using the "Restart & Clear Output" button in the Kernel drop-down box above.

<a id='exercise2'></a>
# Exercise 2: Experimenting with vehicle speed

__GOAL__: Visualize the effect of a static `Planner` configuration on the AV at different driving speeds.

The previous exercise constructed a `Planner` that was able to prevent a collision with the pedestrian at a speed of 12m/s. However, as discussed in the presentation, the speed at which the vehicle is driving requires the vehicle to have different response times in order to remain safe. 

For example, the image below shows that a vehicle driving at 12m/s is able to stop for the pedestrian with a response time of 400ms. However, the same vehicle requires a 200ms response time to be able to prevent a collision at 14m/s.

<img src="risecamp/Stopping-distance.png" alt="Stopping Distance" style="width: 800px;"/>
<center><b>Figure 4:</b> Different driving speeds require different response times. A response time of 400ms is sufficient for an AV driving at 12m/s. However, the AV requires a response time of 200ms at 14m/s in order to ensure safety.</center>

We will now explore the effects of different driving speeds of the AV on its ability to prevent a collision under a static configuration. To achieve this, complete the following tasks:
1. Change the `VEHICLE_SPEED` to a desired speed.
2. Run the following two cells to see the effect of the vehicle speed on its ability to prevent a collision with a static configuration.

In [ ]:
# TODO: Change the vehicle speed here to see the effect. (Try 14m/s!)
VEHICLE_SPEED = 12 # m/s 

# Run the simulator and the scenario (Don't change!)
import pylot.risecamp 
simulator = pylot.risecamp.start_simulator()
scenario = pylot.risecamp.setup_scenario()
print("Please execute the next cell now.")

In [ ]:
# Helper function to start the Pylot pipeline.
video_name = pylot.risecamp.start_pylot(vehicle_speed=VEHICLE_SPEED)

# Functions to reset the scenario and the simulator. (Don't change!)
simulator.reset()
scenario.reset()

# Visualize the drive.
from IPython.display import Video
Video(video_name, html_attributes="loop autoplay")

<b>NOTE:</b> Please wait a few seconds for the above cell to start the visualization. If the visualization does not start within 20 seconds, please restart the kernel using the "Restart & Clear Output" button in the Kernel drop-down box above.

# Exercise 3: Experimenting with different configurations

__GOAL__: Explore different configurations of the `Planner` to prevent a collision at different driving speeds.

<img src="risecamp/fot2.gif" alt="Stopping Distance" style="width: 500px;"/>
<center><b>Figure 5:</b> A trajectory generated by the `Planner` in the presence of obstacles. The grid represents the discretization of the space around the AV, and the red dots represent the points on the generated trajectory that the control module of the AV follows.</center>

The planning module that we built in [Exercise 1](#exercise_1) accounts for the kinematic, dynamic and road shape constraints when producing a feasible trajectory for the AV. In order to find such a trajectory, the planner discretizes both the space around the vehicle (by representing it as a grid as shown in __Figure 5__), and the expected time between two consecutive points on the trajectory. 

A smaller value of both the space and the time discretization ensures a smoother trajectory of the vehicle by generating waypoints that are close to each other, and hence easier for the control module to follow. However, this fine-grained discretization comes at the cost of an increased runtime of the planner which leads to an increased end-to-end response time. On the other hand, the planner might fail to find a feasible trajectory with a coarse-grained discretization of space and time.

[Exercise 2](#exercise_2) showed us that a static configuration of these hyperparameters does not allow the AV to prevent a collison across various driving speeds. To ensure safety of the AV across different driving speeds, [ERDOS](https://github.com/erdos-project/erdos) enables an AV pipeline such as [Pylot](https://github.com/erdos-project/pylot) to adjust the runtime of each component in order to ensure the best accuracy for a given environment. For example, the pipeline might choose to run a faster object-detection model that has a lower accuracy in order to allocate more time to the planning module. This may allow the planner to reduce the discretization of both time and space in order to generate a safe plan for the AV to follow.



We will now explore the effects of the two hyperparameters on the ability of our `Planner` to prevent a collision:
1. __*Space Discretization*__: The discretization of the space around the vehicle in metres. 
2. __*Time Discretization*__: The amount of time between two consecutive points on the trajectory.

To achieve this, please complete the following tasks:
1. Change the `SPACE_DISCRETIZATION` and `TIME_DISCRETIZATION` of the planner to values between 0.1 and 0.5.
2. Run the following two cells to see the effect of your choice of hyperparameters on the AV.

In [ ]:
# TODO: Change the time and space discretization parameters.
SPACE_DISCRETIZATION = 0.35
TIME_DISCRETIZATION = 0.35
VEHICLE_SPEED = 14

# Run the simulator and the scenario (Don't change!)
import pylot.risecamp 
simulator = pylot.risecamp.start_simulator()
scenario = pylot.risecamp.setup_scenario()
print("Please execute the next cell now.")

In [ ]:
# Helper function to start the Pylot pipeline.
video_name = pylot.risecamp.start_pylot(vehicle_speed=VEHICLE_SPEED,
                           time_discretization=TIME_DISCRETIZATION, 
                           road_width=SPACE_DISCRETIZATION)

# Functions to reset the scenario and the simulator. (Don't change!)
simulator.reset()
scenario.reset()

# Visualize the drive.
from IPython.display import Video
Video(video_name, html_attributes="loop autoplay")

<b>NOTE:</b> Please wait a few seconds for the above cell to start the visualization. If the visualization does not start within 20 seconds, please restart the kernel using the "Restart & Clear Output" button in the Kernel drop-down box above.

<b>NOTE:</b> A low value of both `SPACE_DISCRETIZATION` and `TIME_DISCRETIZATION` increases the runtime of the planner, and [Pylot](https://github.com/erdos-project/pylot) might require a considerable amount of time to finish executing the scenario. For your convenience, we have provided the results (along with a video of the AV) for a few choices of the two hyperparameters below:

<table style="width:100%">
  <tr>
      <th><center>Speed [m/sec]</center></th>
    <th><center>Space Discretization [m]</center></th> 
    <th><center>Time Discretization [sec]</center></th>
    <th><center>Prevents Collision?</center></th>
  </tr>
  <tr>
    <td><center>14</center></td>
    <td><center>0.35</center></td> 
    <td><center>0.35</center></td>
    <td><center><details><img src="risecamp/x.png" />Vehicle crashes into the pedestrian.</details></center></td>
  </tr>
  <tr>
    <td><center>14</center></td>
    <td><center>0.30</center></td> 
    <td><center>0.30</center></td>
    <td><center><details><img src="risecamp/x.png" />Vehicle crashes into the pedestrian.</details></center></td>
  </tr>
  <tr>
    <td><center>14</center></td>
    <td><center>0.25</center></td> 
    <td><center>0.25</center></td>
    <td><center><details><img src="risecamp/x.png" />Vehicle crashes into the pedestrian.</details></center></td>
  </tr>
  <tr>
    <td><center>14</center></td>
    <td><center>0.20</center></td> 
    <td><center>0.20</center></td>
    <td><center><details><img src="risecamp/check.png" />Vehicle halts, and then carries on to the destination.</details></center></td>
  </tr>
  <tr>
    <td><center>14</center></td>
    <td><center>0.15</center></td> 
    <td><center>0.15</center></td>
    <td><center><details><img src="risecamp/check.png" />Vehicle smoothly moves away from the pedestrian, and then carries on to the destination.</details></center></td>
  </tr>
</table>

<hr>

### Videos of the AV with different configurations.

`SPACE_DISCRETIZATION` = 0.35, `TIME_DISCRETIZATION` = 0.35, `VEHICLE_SPEED` = 14
<details>
    <summary>Click to show the video</summary>
    <video controls src="video_14_0.35_0.35.mp4"/>
</details>

<hr>

`SPACE_DISCRETIZATION` = 0.3, `TIME_DISCRETIZATION` = 0.3, `VEHICLE_SPEED` = 14
<details>
    <summary>Click to show the video</summary>
    <video controls src="video_14_0.3_0.3.mp4"/>
</details>

<hr>

`SPACE_DISCRETIZATION` = 0.25, `TIME_DISCRETIZATION` = 0.25, `VEHICLE_SPEED` = 14
<details>
    <summary>Click to show the video</summary>
    <video controls src="video_14_0.25_0.25.mp4"/>
</details>

<hr>

`SPACE_DISCRETIZATION` = 0.2, `TIME_DISCRETIZATION` = 0.2, `VEHICLE_SPEED` = 14
<details>
    <summary>Click to show the video</summary>
    <video controls src="video_14_0.2_0.2.mp4"/>
</details>

<hr>

`SPACE_DISCRETIZATION` = 0.15, `TIME_DISCRETIZATION` = 0.15, `VEHICLE_SPEED` = 14
<details>
    <summary>Click to show the video</summary>
    <video controls src="video_14_0.15_0.15.mp4"/>
</details>

# Conclusion

Congratulations on completing the tutorial! To recap, the tutorial showed:
1. How to develop and deploy a Planning component in ERDOS and Pylot.
2. How a static configuration of the space and time discretization is unable to ensure the safety of the AV across different speeds.
3. How a dynamic change in the space and time discretization of the Planner can prevent collisions across speeds.

If you would like to learn more about the [ERDOS](https://github.com/erdos-project/erdos) API, you can go to [this notebook](./ERDOS&#32;&#32;\'20&#32;-&#32;Perception.ipynb) that walks you through the development and deployment of a Perception component in [Pylot](https://github.com/erdos-project/pylot).

### __Finally, please don't forget to fill out the survey [here](https://forms.gle/MxxymmgrfjXACUkM8)!__
